### Install requirements

In [ ]:
%pip install -r requirements.txt

### Load Libraries, Set Global Variables and Settings

In [26]:
from __future__ import print_function
import logging
import sys
import os
import glob
from dotenv import load_dotenv
import logging
import sys
import duckdb
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.core import ( Settings, VectorStoreIndex, SimpleDirectoryReader)
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import (StorageContext, ServiceContext)
from llama_index.core.node_parser import MarkdownNodeParser
import chromadb
import datetime
import numpy as np
from llama_index.core.vector_stores import MetadataFilter, MetadataFilters, ExactMatchFilter
from sqlalchemy import *
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.tools import QueryEngineTool
from llama_index.core.tools import ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core import SQLDatabase, Document
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core.indices.struct_store import SQLTableRetrieverQueryEngine
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.query_engine import SQLJoinQueryEngine
from llama_index.vector_stores.duckdb import DuckDBVectorStore
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.vector_stores import MetadataInfo, VectorStoreInfo
from langchain.text_splitter import RecursiveCharacterTextSplitter
from llama_index.core.node_parser import LangchainNodeParser
from sqlalchemy import (create_engine,MetaData,Table,Column,String,Integer,select,column,)
from sqlalchemy.dialects.postgresql import (INTEGER, FLOAT, BIGINT, VARCHAR, DOUBLE_PRECISION)
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core.indices.struct_store import SQLTableRetrieverQueryEngine
from llama_index.core.objects import (SQLTableNodeMapping,ObjectIndex,SQLTableSchema,)
from llama_index.core import VectorStoreIndex
from llama_index.core.query_engine import SQLAutoVectorQueryEngine
from langchain_text_splitters import RecursiveJsonSplitter
import json
from os import linesep
import json
import pandas as pd
from itertools import chain


# NOTE: This is ONLY necessary in jupyter notebook.
import nest_asyncio

nest_asyncio.apply()

logging.getLogger().setLevel(logging.ERROR)

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

Settings.callback_manager = callback_manager

load_dotenv('/Users/sam/dev/spaghetti_dev/MedEd-AI/Credentials/.env')

#Azure OpenAI Creds
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
credential = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_api_version = "2024-04-01-preview"
azure_openai_embedding_deployment = "text-embedding-ada-002"
embedding_model_name = "text-embedding-ada-002"
llm_model_name = "gpt-35-turbo-16k"
api_type = "azure"

#Google Creds
CREDENTIALS_FILE = '../Credentials/credentials.json'
CLIENT_SECRETS_FILE = '../Credentials/client_secret.json'
SCOPES = ['https://www.googleapis.com/auth/forms.body']
DISCOVERY_DOC = "https://forms.googleapis.com/$discovery/rest?version=v1"

user_path = "Users/sam/dev/spaghetti_dev/"

llm = AzureOpenAI(
            model = llm_model_name,
            deployment_name = llm_model_name,
            api_key = credential,
            azure_endpoint = endpoint,
            api_version = azure_openai_api_version,
            api_type = api_type
        )

embed_model = AzureOpenAIEmbedding(
            model = embedding_model_name,
            deployment_name = embedding_model_name,
            api_key = credential,
            azure_endpoint = endpoint,
            api_version = azure_openai_api_version,
            api_type = api_type,
            embed_batch_size=50
        )

Settings.llm = llm
Settings.embed_model = embed_model

### Connect / Load DuckDB, Define SQL Query Engine

In [2]:
%pip install --upgrade duckdb

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [ ]:
engine = create_engine("duckdb://///Users/sam/dev/spaghetti_dev/MedEd-AI/Data/meded_ai_dev.duckdb", future=True)

metadata_obj = MetaData()

alter_schema = Table('svr_course_quiz_view', 
               metadata_obj, 
               Column("quiz_id", INTEGER), 
               Column('quiz_type', VARCHAR), 
               Column('quiz_title', VARCHAR),
               Column('history_id', BIGINT),
               Column('submission_id', BIGINT),
               Column('student_score', DOUBLE_PRECISION),
                Column('quiz_question_count', BIGINT),
               Column('quiz_points_possible', DOUBLE_PRECISION),
               Column('question_points_possible', DOUBLE_PRECISION),
               Column('answer_points_scored', DOUBLE_PRECISION),
               Column('attempt', BIGINT),
               Column('question_name',VARCHAR),
               Column('question_type', VARCHAR),
               Column('question_text', VARCHAR),
               Column('question_answer', VARCHAR),
               Column('student_answer', VARCHAR),
               autoload_with=engine, 
               extend_existing=True)

with engine.connect() as connection:
    with connection:
        metadata=MetaData()
        my_table=Table("svr_course_quiz_view", metadata, autoload_with=connection)

sql_database = SQLDatabase(engine, include_tables=["svr_course_quiz_view"])

table_node_mapping = SQLTableNodeMapping(sql_database)

table_schema = [SQLTableSchema(table_name='svr_course_quiz_view')]

obj_index = ObjectIndex.from_objects(
    table_schema,
    table_node_mapping,
    VectorStoreIndex,
)

sql_query_engine = SQLTableRetrieverQueryEngine(
    sql_database,
    obj_index.as_retriever(similarity_top_k=1),
)


### Test the SQL Query Engine

In [ ]:
response = sql_query_engine.query("what is the quiz_id for CBB 1 Consolidation Assessment Exercise?")
print(response)

### Connect to ChromaDB, Create Collection

In [2]:
remote_db = chromadb.HttpClient(host='localhost',port=8000)
remote_db.delete_collection("quickstart")
chroma_collection = remote_db.get_or_create_collection("quickstart")

### Load PDF Course Files, Add Metadata, Create Vector Index

In [ ]:
course_quiz_views = [x for x in glob.glob('/Users/sam/dev/spaghetti_dev/MedEd-AI/Data/coursedata_*/course_quiz_view.json',recursive=True)]

for course_quiz_view in course_quiz_views:
    df = pd.read_json(course_quiz_view)
    df = df.loc[(df['question_type'] == 'essay_question') & (df['quiz_title'].str.contains("Consolidation"))]
    path = course_quiz_view.split('/')
    new_path=[path[0],path[1],path[2],path[3],path[4],path[5],'Data','quizviews',path[7][-6:]+'_'+'cleaned_quiz.json']
    new_path = '/'.join(new_path)
    df.to_json(new_path, 'records')
    

In [ ]:
[x for x in glob.glob('/Users/sam/dev/spaghetti_dev/MedEd-AI/Data/quizviews/*_cleaned_quiz.json',recursive=True)]

In [46]:
logging.getLogger().setLevel(logging.WARNING)
pd.set_option('future.no_silent_downcasting', True)
# embed that content, with metadata for where they came from/what consolidation exercise they're a part of 

documents = []

# # process to get cleaned quizzes into text files
# cleaned_quizzes = [x for x in glob.glob('/Users/sam/dev/spaghetti_dev/MedEd-AI/Data/graded_quizzes/graded_quizzes.json',recursive=True)]

# cleaned_quiz = cleaned_quizzes[0]

# json_df = pd.read_json(cleaned_quiz)

# quiz_titles = list(set(json_df['quiz_title'].values))

# for title in quiz_titles:
#     filtered_json_df = json_df.loc[json_df['quiz_title'] == title]
#     title_path = title.replace(" ", "_")
#     quiz_id = str(list(set(filtered_json_df['quiz_id'].values))[0])   
#     questions = list(set(filtered_json_df['question_name'].values))
#     questions.sort()
#     for question in questions:
#         question_name=question.replace(' ','_')
#         with open(f"/Users/sam/dev/spaghetti_dev/MedEd-AI/Data/studentguides/{quiz_id+'_'+str(list(set(filtered_json_df['course_id'].values))[0])+'_'+title_path+'_'+question_name}.txt", "a") as f:
#             f.write(f'Quiz Title: {title}') 
#             f.write(f"\n{question}: {filtered_json_df.loc[(filtered_json_df['quiz_title'] == title) & (filtered_json_df['question_name'] == question)]['question_text'].iat[0].replace('&nbsp;',' ')}")    
#             try:                
#                 f.write(f"\nCorrect Answer: {filtered_json_df.loc[(filtered_json_df['quiz_title'] == title) & (filtered_json_df['question_name'] == question)]['question_answer'].iat[0].replace('Answer: ','').replace('&nbsp;',' ')}")    
#             except:
#                 f.write("\nCorrect Answer: ''")
#             student_responses = filtered_json_df.loc[(filtered_json_df['quiz_title'] == title) & (filtered_json_df['question_name'] == question)]['student_answer'].to_list()
#             student_ids = filtered_json_df.loc[(filtered_json_df['quiz_title'] == title) & (filtered_json_df['question_name'] == question)]['submission_id'].to_list()
#             for i in range(0,len(student_responses)):
#                 try:
#                     f.write(f"\nStudent Answer {student_ids[i]}: {student_responses[i].replace(linesep,'').replace('&nbsp;',' ')}")
#                 except:
#                     pass

reader = SimpleDirectoryReader("/Users/sam/dev/spaghetti_dev/MedEd-AI/Data/studentguides/", recursive=True, filename_as_id=True, required_exts=[".txt",".pdf", ".docx", ".xlsx", ".pptx"])

for docs in reader.iter_data():
    if docs[0].metadata['file_name'].endswith('.txt'):
        for doc in docs:
            quiz_data = '/Users/sam/dev/spaghetti_dev/MedEd-AI/Data/graded_quizzes/graded_quizzes.json'
            quiz_df = pd.read_json(quiz_data)     
            filtered_quiz_df = quiz_df.loc[quiz_df['quiz_id'] == int(docs[0].metadata['file_name'][0:6])]
            doc.metadata.update({'quiz_id': docs[0].metadata['file_name'][0:6], 'course_id': str(list(set(filtered_quiz_df['course_id'].values))[0]), 'quiz_type': 'graded_survey','quiz_title': ' '.join(docs[0].metadata['file_name'].split('_')[2:]).replace('.txt','')})
    else:        
        file_filename = [x for x in glob.glob('/Users/sam/dev/spaghetti_dev/MedEd-AI/Data/coursedata_*/course_file.json',recursive=True) if docs[0].metadata['file_name'][0:6] in x][0]
        file_df = pd.read_json(file_filename)
        file_metadata = file_df.loc[file_df['filename'] == docs[0].metadata['file_name'][7:].replace('_','+')]
        if file_metadata.empty != True:                
            file_metadata = file_metadata.squeeze().to_dict()
            file_metadata = pd.DataFrame(file_metadata, index=[0]).replace(np.NaN, 0).replace(0, None)
            file_metadata = file_metadata.to_dict('records')[0]
            folder = file_metadata.get('folder_id')
        else:
            file_metadata = {}
            folder = ''

        folder_filename = [x for x in glob.glob('/Users/sam/dev/spaghetti_dev/MedEd-AI/Data/coursedata_*/course_folder.json',recursive=True) if docs[0].metadata['file_name'][0:6] in x][0]
        folder_df = pd.read_json(folder_filename)
        folder_metadata = folder_df.loc[folder_df['id'] == folder]
        if folder_metadata.empty != True:
            folder_metadata = folder_metadata.squeeze().to_dict()
            folder_metadata = pd.DataFrame(folder_metadata, index=[0]).replace(np.NaN, 0).replace(0, None)
            folder_metadata = folder_metadata.to_dict('records')[0]
            if 'Week' in folder_metadata['full_name']:
                week = [i for i in folder_metadata['full_name'].split("/") if 'Week' in i][0].replace('Week','').replace(' ','')
                folder_metadata.update({"week":week})
            full_name = folder_metadata['full_name'].split("/")[-1]
            folder_metadata.update({"folder_name":full_name})
        else:
            folder_metadata = {}

        course_filename = [x for x in glob.glob('/Users/sam/dev/spaghetti_dev/MedEd-AI/Data/coursedata_*/course_course.json',recursive=True) if docs[0].metadata['file_name'][0:6] in x][0]
        course_df = pd.read_json(course_filename)        
        course_metadata = course_df.loc[course_df['id'] == folder_metadata.get('context_id')]
        if course_metadata.empty != True:
            course_metadata = course_metadata.squeeze().to_dict()
            course_id = folder_metadata.get('context_id')
        else:
            course_metadata = {}

        for doc in docs:
            doc.metadata.update({"file_id": file_metadata.get('id'), "folder_id":file_metadata.get('folder_id'), "display_name":file_metadata.get('display_name')})
            doc.metadata.update({"week": folder_metadata.get('week'),  "folder_name": folder_metadata.get('folder_name')})
            doc.metadata.update({"course_id": course_metadata.get('id'), "course_name":course_metadata.get('name'),"course_code":course_metadata.get('course_code'),"course_term":course_metadata.get('term', {}).get('name')}) 

    documents.extend(docs)

parser = LangchainNodeParser(RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
))

nodes = parser.get_nodes_from_documents(documents)

# vector_store = DuckDBVectorStore(nodes=nodes)
vector_store = DuckDBVectorStore()

storage_context = StorageContext.from_defaults(vector_store=vector_store)

vector_index = VectorStoreIndex(nodes, storage_context=storage_context, embed_model=embed_model, llm=llm)

**********
Trace: index_construction
    |_embedding -> 1.024633 seconds
    |_embedding -> 0.949577 seconds
    |_embedding -> 0.904948 seconds
    |_embedding -> 0.737422 seconds
    |_embedding -> 0.575504 seconds
    |_embedding -> 0.536369 seconds
    |_embedding -> 0.520297 seconds
    |_embedding -> 0.71888 seconds
    |_embedding -> 0.657871 seconds
    |_embedding -> 0.759617 seconds
    |_embedding -> 0.610746 seconds
    |_embedding -> 0.617601 seconds
    |_embedding -> 0.560672 seconds
    |_embedding -> 0.498974 seconds
    |_embedding -> 0.638788 seconds
    |_embedding -> 0.621618 seconds
    |_embedding -> 0.568498 seconds
    |_embedding -> 0.628898 seconds
    |_embedding -> 0.584668 seconds
    |_embedding -> 0.560071 seconds
    |_embedding -> 0.719984 seconds
    |_embedding -> 0.716676 seconds
    |_embedding -> 0.567094 seconds
    |_embedding -> 0.5751 seconds
    |_embedding -> 0.531464 seconds
    |_embedding -> 0.488388 seconds
    |_embedding -> 0.588266 se

In [47]:
docs[0].metadata

{'file_path': '/Users/sam/dev/spaghetti_dev/MedEd-AI/Data/studentguides/350093_130681_Clinic_9_-_Immune_Side_Effects_from_Cancer_Immunotherapy_Consolidation_Assessment_Exercise_Question_3.txt',
 'file_name': '350093_130681_Clinic_9_-_Immune_Side_Effects_from_Cancer_Immunotherapy_Consolidation_Assessment_Exercise_Question_3.txt',
 'file_type': 'text/plain',
 'file_size': 34182,
 'creation_date': '2024-07-01',
 'last_modified_date': '2024-07-01',
 'quiz_id': '350093',
 'course_id': '130681',
 'quiz_type': 'graded_survey',
 'quiz_title': 'Clinic 9 - Immune Side Effects from Cancer Immunotherapy Consolidation Assessment Exercise Question 3'}

### Check Metadata

In [48]:
query = "How many consolidation assessment exercises are there, and what are they called?"
query_engine = vector_index.as_query_engine()
response = query_engine.query(query)
print(response)

**********
Trace: query
    |_query -> 3.720691 seconds
      |_retrieve -> 2.384825 seconds
        |_embedding -> 0.599231 seconds
      |_synthesize -> 1.335 seconds
        |_templating -> 9e-06 seconds
        |_llm -> 1.326589 seconds
**********
There are two consolidation assessment exercises mentioned in the context. One is called "Anatomy 6 Consolidation Assessment Exercise Question 1" and the other is called "Pathology 4 Consolidation Assessment Exercise Question 5".


### Define RetrieverQueryEngine

In [ ]:
vector_store_info = VectorStoreInfo(
    content_info="Student guides to help prepare for consolidation assessments",
    metadata_info=[
        MetadataInfo(
            name="page_label",
            description="What page of the file the context is from",
            type="string",
        ),
        MetadataInfo(
            name="file_name",
            description="The name of the file the context is from",
            type="string",
        ),
        MetadataInfo(
            name="file_path",
            description="The file path of the context file",
            type="string",
        ),
        MetadataInfo(
            name="file_type",
            description="The type of file",
            type="string",
        ),
        MetadataInfo(
            name="file_size",
            description="The size of the file in bytes",
            type="integer",
        ),
        MetadataInfo(
            name="creation_date",
            description="When the file was created",
            type="string",
        ),
        MetadataInfo(
            name="last_modified_date",
            description="When the file was last modified",
            type="string",
        ),
        MetadataInfo(
            name="display_name",
            description="The name of the file",
            type="string",
        ),
        MetadataInfo(
            name="week",
            description="The week the context was administered",
            type="string",
        ),
        MetadataInfo(
            name="folder_name",
            description="The course folder that contains the file",
            type="string",
        ),
        MetadataInfo(
            name="course_id",
            description="The unique identifier of the course",
            type="integer",
        ),
        MetadataInfo(
            name="course_name",
            description="The full name of the course",
            type="string",
        ),
        MetadataInfo(
            name="course_code",
            description="The shortened name of the course",
            type="string",
        ),
        MetadataInfo(
            name="course_term",
            description="What term the course was offered in",
            type="string",
        ),
    ],
)

vector_auto_retriever = VectorIndexAutoRetriever(
    vector_index, vector_store_info=vector_store_info
)

retriever_query_engine = RetrieverQueryEngine.from_args(
    vector_auto_retriever, llm=llm
)

### Set Up Query Engine Tools

In [ ]:
from llama_index.core.tools import QueryEngineTool

sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    description=(
        "Useful for translating a natural language query into a SQL query over"
        " a table svr_course_quiz_view, containing columns:"
        " quiz_id, quiz_type, quiz_title, history_id, submission_id,"
        " student_score, quiz_question_count, quiz_points_possible, question_points_possible," 
        "answer_points_scored, attempt, question_name, question_type, question_text, question_answer, and student_answer"
    ),
)


vector_tool = QueryEngineTool.from_defaults(
        query_engine=retriever_query_engine,
        description=f"Useful for answering semantic questions about consolidation assessments, and general course-related questions like when certain material is being taught",
    )


### Define Query Engine

In [ ]:
query_engine = SQLAutoVectorQueryEngine(
    sql_tool, 
    vector_tool,
    llm=llm
)

### Query Test 1: Ability to identify topics covered in a specific assessment

In [45]:
response = query_engine.query("What is in the context?")

response.response

**********
Trace: query
    |_query -> 2.643768 seconds
      |_retrieve -> 0.484422 seconds
        |_embedding -> 0.287008 seconds
      |_synthesize -> 2.158525 seconds
        |_templating -> 9e-06 seconds
        |_llm -> 2.147489 seconds
**********


'The context information includes details about two different files. The first file is named "334985_130681_Clinic_1_-_Down_Syndrome_Consolidation_Assessment_Exercise__Question_2.txt" and the second file is named "338474_130681_Clinic_2_-_Sickle_Cell_Anemia_Consolidation_Assessment_Exercise__Question_1.txt". The context also provides information about the file type, file size, creation date, last modified date, quiz ID, course ID, quiz type, and quiz title for each file. Additionally, there are student answers provided for each file.'

### Query Test 2: Ability to query SQL db, synthesize results and deliver a cohesive answer

In [39]:
response = query_engine.query("Compare all student answers to the answer for question 1 of CBB 3 Consolidation Assessment Exercise. What concept did students best understand? Which concept was most frequently not mentioned?")

response.response += ' '
out = response.response.split('. ')[:-1]
print('.\n'.join(out))

**********
Trace: query
    |_query -> 1.510875 seconds
      |_retrieve -> 0.459248 seconds
        |_embedding -> 0.271059 seconds
      |_synthesize -> 1.050958 seconds
        |_templating -> 7e-06 seconds
        |_llm -> 1.043321 seconds
**********
Based on the given context information, it is not possible to determine the specific concepts that students best understood or the concept that was most frequently not mentioned in question 1 of the CBB 3 Consolidation Assessment Exercise.
The context information does not provide any information about the student answers or the specific concepts covered in the exercise


### Query Test 3: Ability to identify material from specific time, create quiz questions, create Google quiz

In [ ]:

response = query_engine.query(
    'Identify what material is taught in week 7. Then create a 5 question review quiz on the material taught in week 7. Do not ask questions about text from a photo/graph. Do not ask any questions about textbook sections. Do not ask any questions about learning objectives. Every question should have only one correct answer. Every question should have 4 potential answers. Format the output like this:{"info":{"title": "", "description": "", "questions": [ { "question": "","type": "RADIO","options": [], "correct_answer": ""}]}}'
)

# Google Quiz Creation
form = Form(file_type='credentials', loginfile=CREDENTIALS_FILE,
        discovery_doc=DISCOVERY_DOC, scopes=SCOPES, sentence_response = response)


### Chat Session Testing

In [ ]:
# response = query_engine.query(
#     'Identify what material is taught in week 7. Only respond with the topics, separated with commas.'
# )

# response.response

response = query_engine.query(
    'Create a 6 question review quiz on the hormonal Control of Metabolism. Do not ask questions about text from a photo/graph. Do not ask any questions about textbook sections. Do not ask any questions about learning objectives. Every question should have only one correct answer. Every question should have 4 potential answers. 2 of the questions should be easy in difficulty. 2 of the questions should be medium in difficulty. 2 of the questions should be hard in difficulty. Format the output like this:{"info":{"title": "", "description": "", "questions": [ { "question": "","type": "RADIO","options": [], "correct_answer": ""}]}}'
)

# Google Quiz Creation
form = Form(file_type='credentials', loginfile=CREDENTIALS_FILE,
        discovery_doc=DISCOVERY_DOC, scopes=SCOPES, sentence_response = response)